In [1]:
import pandas as pd
import spacy
import os
import re

In [2]:
vf_files = [
    'VF_Argonautica_1.csv',
    'VF_Argonautica_2.csv',
    'VF_Argonautica_3.csv',
    'VF_Argonautica_4.csv',
    'VF_Argonautica_5.csv',
    'VF_Argonautica_6.csv',
    'VF_Argonautica_7.csv',
    'VF_Argonautica_8.csv',
]

nlp = spacy.load('la_core_web_lg')

In [3]:
tables = []
cur_id = 7000

for file in vf_files:
    print(file)
    df = pd.read_csv(os.path.join('data', 'vf', file))

    speech_ids = []
    last_spkr = None
    for speaker in df.speaker:
        if pd.isna(speaker):
            id = None
        else:
            if speaker != last_spkr:
                cur_id += 1
            id = cur_id
        last_spkr = speaker
        speech_ids.append(id)
        
    df['speech'] = speech_ids
    df = df.loc[~df.speaker.isna()]
    df = df.loc[~df.perseus_text.isna()]
    df = df.loc[~df.pc_bounds.isna()]    
    df['speech'] = df['speech'].astype(int)
    df['file'] = file[:-4]
    df['line_id'] = df['file'] + ':' + df['perseus_n'].astype(str)
    df['speaker'] = df['speaker'].str.replace('[\[\]\']', '', regex=True)
    df['tokens'] = df['perseus_text'].apply(lambda s: [tok for tok in nlp(s)])

    tables.append(df)

df = pd.concat(tables)
df['line_id'] = pd.Categorical(df['line_id'], categories=pd.unique(df['line_id']), ordered=True)

VF_Argonautica_1.csv
VF_Argonautica_2.csv
VF_Argonautica_3.csv
VF_Argonautica_4.csv
VF_Argonautica_5.csv
VF_Argonautica_6.csv
VF_Argonautica_7.csv
VF_Argonautica_8.csv


In [4]:
def normalize(s):
    return re.sub(r'[^a-z]', '', s.lower()).replace('jv', 'iu')


def getElided(df):
    '''extract elided tokens from a line-array table'''
    rows = []
    row_count = 0
    
    for row in df.itertuples():
        elided = [False] * len(row.tokens)
        row_count += 1
        
        if 'SY' in row.pc_bounds:
            bounds = row.pc_bounds[1:-1].split(',')
            pc_toks = row.pc_text.split()
            no_punct = [tok for tok in row.tokens if tok.pos_ != 'PUNCT']
            
            if len(no_punct) == len(bounds):
                for bound, tok in zip(bounds, no_punct):
                    if 'SY' in bound:
                        idx = row.tokens.index(tok)
                        elided[idx] = True
            else:
                if len(bounds) == len(pc_toks):
                    for bound, pc_tok in zip(bounds, pc_toks):
                        if 'SY' in bound:
                            normalized_pc = normalize(pc_tok)
                            normalized_toks = [normalize(tok.text) for tok in row.tokens]
                            count = normalized_toks.count(normalized_pc)
                            if count == 1:
                                idx = normalized_toks.index(normalized_pc)
                            elif pc_tok.endswith('que') and normalized_toks.count('que') == 1:
                                idx = normalized_toks.index('que')
                            else:
                                print(f'[{row_count}]\t' + ' '.join([f'{i}.{tok.text}' for i, tok in enumerate(row.tokens)]))
                                print(' '.join(bounds))
                                idx = int(input(f'Which word is {pc_tok}? '))
                            elided[idx] = True
        rows.append(elided)
    return rows

In [5]:
# df['elided'] = getElided(df)

[24]	0.da 1.Scythiam 2.Phasim 3.que 4.mihi 5.; 6.tu 7.que 8., 9.innuba 10.Pallas 11.,
'CM'  'CM'  'CF'  'CM'  'SY'  'DI'  None


Which word is tuque,?  7


[31]	0.iam 1.iam 2.ego 3.et 4.inviti 5.torsissem 6.coniugis 7.ignem 8..
'SY'  'SY'  'DI'  'CM'  'DI'  'DI'  None


Which word is Iamiam?  1


[38]	0.non 1.iuvenem 2.in 3.casus 4.eademn 5.que 6.pericula 7.Acastum
'CM'  'SY'  'CM'  'CM'  'CF'  'SY'  None


...

Which word is meminique,?  5


[1879]	0.me 1.cum 2.adsunt 3.. 4.magni 5.virgo 6.ne 7.regia 8.Solis
'SY'  'CM'  'CM'  'CM'  'DI'  'DI'  None


Which word is Mecum?  1


[1911]	0.te 1.que 2.simul 3.me 4.cum 5.ipsa 6.traham 7.; 8.non 9.sola 10.reposcor
'CF'  'CM'  'SY'  'CF'  'CM'  'DI'  'CF'  None


Which word is mecum?  4


In [6]:
df.to_csv(os.path.join('data','vf_scanned.csv'), index=False)
df

,comp,perseus_n,perseus_text,pc_n,pc_text,pc_bounds,elision,speaker,speech,file,line_id,tokens,elided
39,0.931818,40,"'hanc mihi militiam, veterum quae pulchrior ac...",40.0,"""Hanc mihi militiam, ueterum quae pulchrior ac...","['CM', 'DI', 'CM', 'CM', 'DI', 'DI', None]",0,Pelias,7001,VF_Argonautica_1,VF_Argonautica_1:40,"[', hanc, mihi, militiam, ,, veterum, quae, pu...","[False, False, False, False, False, False, Fal..."
40,1.000000,41,adnue daque animum. nostri de sanguine Phrixus,41.0,Adnue daque animum. nostri de sanguine Phrixus,"['DI', 'SY', 'CM', 'CM', 'DI', 'DI', None]",1,Pelias,7001,VF_Argonautica_1,VF_Argonautica_1:41,"[adnue, da, que, animum, ., nostri, de, sangui...","[False, False, True, False, False, False, Fals..."
41,1.000000,42,Cretheos ut patrias audis effugerit aras.,42.0,Cretheos ut patrias audis effugerit aras.,"['DI', 'CM', 'CM', 'CM', 'DI', None]",0,Pelias,7001,VF_Argonautica_1,VF_Argonautica_1:42,"[Cretheos, ut, patrias, audis, effugerit, aras...","[False, False, False, False, False, False, False]"
42,0.928571,43,"hunc ferus Aeetes, Scythiam Phasimque rigentem",43.0,"Hunc ferus Aeetes, Scythiam Phasinque rigentem","['CM', 'DI', 'CM', 'CM', 'CF', None]",0,Pelias,7001,VF_Argonautica_1,VF_Argonautica_1:43,"[hunc, ferus, Aeetes, ,, Scythiam, Phasim, que...","[False, False, False, False, False, False, Fal..."
43,0.926829,44,"qui colit (heu magni Solis pudor), hospita vina",44.0,"Qui colit (heu magni Solis pudor), hospita uina","['CM', 'DI', 'CM', 'CM', 'CM', 'DI', 'DI', None]",0,Pelias,7001,VF_Argonautica_1,VF_Argonautica_1:44,"[qui, colit, (, heu, magni, Solis, pudor, ), ,...","[False, False, False, False, False, False, Fal..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
443,0.954545,441,"nescio quid tuus iste pudor? mene, optime quondam",441.0,"Nescioquid tuus iste pudor? mene, optime quondam","['CM', 'DI', 'CF', 'CM', 'SY', 'DI', None]",1,Medea,7194,VF_Argonautica_8,VF_Argonautica_8:441,"[nescio, quid, tuus, iste, pudor, ?, mene, ,, ...","[False, False, False, False, False, False, Tru..."
444,1.000000,442,"Aesonide, me ferre preces et supplicis ora",442.0,"Aesonide, me ferre preces et supplicis ora","['CM', 'DI', 'CF', 'CM', 'DI', 'DI', None]",0,Medea,7194,VF_Argonautica_8,VF_Argonautica_8:442,"[Aesonide, ,, me, ferre, preces, et, supplicis...","[False, False, False, False, False, False, Fal..."
445,1.000000,443,fas erat? haud hoc nunc genitor putat aut dare...,443.0,Fas erat? haud hoc nunc genitor putat aut dare...,"['CM', 'DI', 'CM', 'DI', 'CM', 'CM', 'DI', 'CM...",0,Medea,7194,VF_Argonautica_8,VF_Argonautica_8:443,"[fas, erat, ?, haud, hoc, nunc, genitor, putat...","[False, False, False, False, False, False, Fal..."
446,1.000000,444,iam sceleris dominumque pati.' sic fata parantem,444.0,"Iam sceleris dominumque pati."" sic fata parantem","['CM', 'CM', 'CF', 'CM', 'DI', 'CF', None]",0,Medea,7194,VF_Argonautica_8,VF_Argonautica_8:444,"[iam, sceleris, dominum, que, pati, ., ', sic,...","[False, False, False, False, False, False, Fal..."


In [7]:
def getContext(token_table):
    df = (token_table
        .assign(lemma = token_table.lemma.str.lower())
        .groupby('line_id', as_index = False)
        .agg(line_id = ('line_id', 'first'), lemmas = ('lemma', list))
    )

    r_context = (
        pd.concat(
            pd.DataFrame(dict(
                line_id = df.iloc[:-i].line_id.values,
                lemmas = df.iloc[i:].lemmas.values,
            )) for i in range(1, 3))
        .groupby('line_id', as_index=False)
        .agg(r_context=('lemmas', lambda lems: sum(lems, [])))
    )

    l_context = (
        pd.concat(
            pd.DataFrame(dict(
                line_id = df.iloc[i:].line_id.values,
                lemmas = df.iloc[:-i].lemmas.values,
            )) for i in range(1, 3))
        .groupby('line_id', as_index=False)
        .agg(l_context=('lemmas', lambda lems: sum(lems, [])))
    )

    context = pd.merge(l_context, r_context, how='outer', on='line_id')
    context = pd.merge(df, context, on='line_id')
    context['context'] = context['l_context'] + context['lemmas'] + context['r_context']
    context = context.drop(columns=['l_context', 'lemmas', 'r_context'])

    return context

In [8]:
tables = []
for label, group in df.groupby('speech'):
    print(label)
    token_table = group.explode(['tokens', 'elided']).rename(columns={'tokens':'token'})
    fulltext = ' '.join(group.perseus_text)
    token_table['token'] = [tok for tok in nlp(fulltext)]
    token_table['lemma'] = [tok.lemma_ for tok in token_table.token]
    context = getContext(token_table)
    reps = pd.merge(token_table, context, how='left', on='line_id')[['lemma', 'context']]
    reps['lemma'] = reps['lemma'].str.lower()
    token_table['reps'] = reps.apply(lambda row: row['context'].count(row['lemma']), axis=1).values

    tables.append(token_table)

7001
7002
7003


/var/folders/2x/rfhzgyjn2zq2wlzk_kn4th680000gn/T/ipykernel_9410/2342058334.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df = (token_table
/var/folders/2x/rfhzgyjn2zq2wlzk_kn4th680000gn/T/ipykernel_9410/2342058334.py:9: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  pd.concat(
/var/folders/2x/rfhzgyjn2zq2wlzk_kn4th680000gn/T/ipykernel_9410/2342058334.py:19: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  pd.concat(
/v

7004
7005
7006
7007


/var/folders/2x/rfhzgyjn2zq2wlzk_kn4th680000gn/T/ipykernel_9410/2342058334.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df = (token_table
/var/folders/2x/rfhzgyjn2zq2wlzk_kn4th680000gn/T/ipykernel_9410/2342058334.py:9: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  pd.concat(
/var/folders/2x/rfhzgyjn2zq2wlzk_kn4th680000gn/T/ipykernel_9410/2342058334.py:19: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  pd.concat(
/v

7008
7009
7010
7011


...

/var/folders/2x/rfhzgyjn2zq2wlzk_kn4th680000gn/T/ipykernel_9410/2342058334.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df = (token_table
/var/folders/2x/rfhzgyjn2zq2wlzk_kn4th680000gn/T/ipykernel_9410/2342058334.py:9: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  pd.concat(
/var/folders/2x/rfhzgyjn2zq2wlzk_kn4th680000gn/T/ipykernel_9410/2342058334.py:19: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  pd.concat(
/v

7189
7190
7191


/var/folders/2x/rfhzgyjn2zq2wlzk_kn4th680000gn/T/ipykernel_9410/2342058334.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df = (token_table
/var/folders/2x/rfhzgyjn2zq2wlzk_kn4th680000gn/T/ipykernel_9410/2342058334.py:9: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  pd.concat(
/var/folders/2x/rfhzgyjn2zq2wlzk_kn4th680000gn/T/ipykernel_9410/2342058334.py:19: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  pd.concat(
/v

7192
7193
7194
7195


/var/folders/2x/rfhzgyjn2zq2wlzk_kn4th680000gn/T/ipykernel_9410/2342058334.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df = (token_table
/var/folders/2x/rfhzgyjn2zq2wlzk_kn4th680000gn/T/ipykernel_9410/2342058334.py:9: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  pd.concat(
/var/folders/2x/rfhzgyjn2zq2wlzk_kn4th680000gn/T/ipykernel_9410/2342058334.py:19: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  pd.concat(
/v

In [9]:
token_table = pd.concat(tables)

token_table['upos'] = [tok.pos_ for tok in token_table.token]
token_table['morph'] = [tok.morph.to_dict() for tok in token_table.token]
token_table['mood'] = [morph.get('Mood') for morph in token_table.morph]
token_table['tense'] = [morph.get('Tense') for morph in token_table.morph]
token_table['voice'] = [morph.get('Voice') for morph in token_table.morph]
token_table['person'] = [morph.get('Person') for morph in token_table.morph]
token_table['number'] = [morph.get('Number') for morph in token_table.morph]
token_table['case'] = [morph.get('Case') for morph in token_table.morph]
token_table['gender'] = [morph.get('Gender') for morph in token_table.morph]
token_table = token_table.drop(columns=['morph', 'perseus_text', 'pc_text', 'pc_bounds'])
token_table = token_table.loc[token_table.upos != 'PUNCT']
token_table.to_csv(os.path.join('data', 'vf_tokens.csv'), index=False)